In [2]:
Output = ('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1012. Allostatic Load/P1012.3. Analyses/P1012.3.2. Ordinal Regression/Output')
cur_date = "052124"

library(readxl)
library(openxlsx)
library(tidyverse)

# reading in file
cytokine_df = data.frame(read_excel("Input/Allostatic_Mediator_Data_050824.xlsx", sheet = 2)) 
bp_df = data.frame(read_excel("Input/Allostatic_Mediator_Data_050824.xlsx", sheet = 4)) 
subject_info_df = data.frame(read_excel("Input/Subject_Info_050824.xlsx", sheet = 2))
subject_info_df = data.frame(read_excel("Input/Subject_Info_050824.xlsx", sheet = 2))
allostatic_load_df = data.frame(read_excel("Input/Allostatic_Load_BP_Subjects_051324.xlsx"))

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.3     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Warning message:
“Expecting numeric in D1077 / R1077C4: got 'NA'”
Warning message:
“Expecting numeric in D1080 / R1080C4: got 'NA'”
Warning message:
“Expecting numeric in D1083 / R1083C4: got 'NA'”
Warning message:
“Expecting numeric in D1091 / R1091C4: got 'NA'”
Warning message:
“Expecting numeric in D1097 / R1097C4: got 'NA'”
Warning message:
“Expecting numeric in D1098 / R1098C4: got 'NA'”
Warning message:
“Expecting numeric in D1114 / R1114C4: got 'NA'”
Warning messag

In [19]:
# creating 1 df
full_df = inner_join(subject_info_df, cytokine_df) %>%
    # filtering for subjects who have blood pressure measurements
    filter(Subject_ID %in% unique(bp_df$Subject_ID)) %>%
    # adding allostatic load data
    inner_join(allostatic_load_df %>%
    # only keep overall allostatic load scores   
    filter(if_any(Covariate, is.na))) %>%
    select(-Covariate)

head(full_df)

Joining with `by = join_by(Subject_ID)`
Joining with `by = join_by(Group, Subject_ID)`


,Study,Original_Subject_Number,Subject_Number,Group,Subject_ID,Sex,Age,Race,Category,Variable,Value,Allostatic_Load
,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>
1,TCORS LAIV,39,5,CS,CS_5,M,21,W,AL Biomarker,Cortisol,116.602,1.492657
2,TCORS LAIV,39,5,CS,CS_5,M,21,W,AL Biomarker,Noradrenaline,6214.280,1.492657
3,TCORS LAIV,39,5,CS,CS_5,M,21,W,AL Biomarker,Hba1c,8901.778,1.492657
4,TCORS LAIV,39,5,CS,CS_5,M,21,W,AL Biomarker,Fibrinogen,1106446.956,1.492657
5,TCORS LAIV,39,5,CS,CS_5,M,21,W,AL Biomarker,CRP,896782.493,1.492657
6,TCORS LAIV,39,5,CS,CS_5,M,21,W,Cytokine,IP10,123.031,1.492657


knownData is a data frame containing the sample names under the first column, the feature data in the next n-1 columns, and the response data in the last column under response. This order matters for line 25, where you get a matrix of just the features. Could be obtained in better ways.
Line 27 runs the actual regression using the matrix of features, and a factor of responses, both having the same order of samples they were obtained from.

In [ ]:
################ Create Required Functions ###################################

#split into 2 functions ord and ga
############## Start Weight Estimation Process #####################
estimateOrdWeights <- function(knownData, model){
  ################# Perform Ordinal Regression ################################
  
  ############# get required Regression Parameters

  #get number of response levels
  NoBins <- length(unique(knownData$response))
  #get number of slices
  noSlices <- length(names(model))
  
  #set ordering of response levels as factor
  orderedBins <- vector()
  for(i in 1:NoBins){
    orderedBins <- c(orderedBins, as.character(i))
  }
  Bins_factor <- factor(knownData[,'response'], ordered = TRUE, levels = orderedBins)
  trues <- rep(TRUE, noSlices)
  
  #run the regression
  tmpData <- as.matrix(knownData[,c(2:(ncol(knownData)-1))])
  class(tmpData) <- "numeric"
  result <- ordinalNet(tmpData, Bins_factor, positiveID = trues, lambdaVals = 0, maxiterOut = 100)
  testW <- result$coefs[NoBins:(noSlices+NoBins-1)]
  testW <- testW/sum(testW)

  return(testW)
}

In [ ]:
############## Create Testing Function #########################################
testWeights <- function(data, knownData, w, observedResponse, binRanks){
  idName <- 'Name'
  tmpData <- as.matrix(data[,-1])
  class(tmpData) <- "numeric"
  score <- rowSums(tmpData*rep(w, each = NROW(tmpData)), na.rm = TRUE)
  testData <- cbind(data,score)
  
  #sort the data based on new scores
  testData <- testData[order(testData[,'score'], decreasing = TRUE),]
  
  #determine the rank and responses of "known" profiles using the proposed weights
  noSamples <- nrow(data)
  noKnown <- nrow(knownData)
  rank <- c(1:noSamples)
  testData <- cbind(testData, rank)
  testData <- cbind(testData, observedResponse)
  testKnowns <- testData[testData[,idName] %in% knownData[,idName],]
  
  #get fitness of weights based on deviation of bins*(indices from bin) for known profiles
  deviation <- 0
  for(i in 1:noKnown){
    #get sample info
    id <- testKnowns[i,idName]
    trueBin <- knownData[knownData[,idName] == id,'response']
    observedBin <- testKnowns[testKnowns[,idName] == id,'observedResponse']
    observedRank <- testKnowns[testKnowns[,idName] == id,'rank']
    #get rank distance of sample from true bin
    if(trueBin < observedBin){
      binDist <- abs(binRanks[trueBin+1] - observedRank - 1)
    } else if(trueBin > observedBin){
      binDist <- binRanks[trueBin] - observedRank
    } else {
      binDist <- 0
    }
    #add sample deviations together
    deviation <- deviation + abs(trueBin - observedBin)*binDist
  }
  return(deviation)
}